# Order Driven Reward
The idea continue from the contour approach, adding more reinforcement learning element like rewards.
Reinforcement learning contains three components:
- State: vector extracted from market history, $S_t=[s_1, s_2,..., s_p]$
- Action: [Buy, Sell]
- Reward: There are several definitions of rewards, it's the essential part of this notebook

In [5]:
# Run it when crypto-backtest is updated, and then restart the kernel
!pip uninstall crypto-backtest -y
!pip install git+https://github.com/BullionBear/crypto-backtest.git

Found existing installation: crypto-backtest 0.1.2
Uninstalling crypto-backtest-0.1.2:
  Successfully uninstalled crypto-backtest-0.1.2
  Cloning https://github.com/BullionBear/crypto-backtest.git to /tmp/pip-req-build-69tu6q_l
  Running command git clone --filter=blob:none --quiet https://github.com/BullionBear/crypto-backtest.git /tmp/pip-req-build-69tu6q_l
  Resolved https://github.com/BullionBear/crypto-backtest.git to commit c88e2b9e9bb881b1991a367735c5c8e36bc7dc0c
  Preparing metadata (setup.py) ... done
  Created wheel for crypto-backtest: filename=crypto_backtest-0.1.3-py3-none-any.whl size=11581 sha256=19f6d2110534c33e7f05710b40c8547ca97f2c744f8ecfcbd8b8f1ed1c794d08
  Stored in directory: /tmp/pip-ephem-wheel-cache-t54zlivj/wheels/14/d1/5f/193347d44b51670dd287000ac535cdb7a8fc3f84e426532382
Successfully built crypto-backtest

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
import gc
gc.collect()

426

In [2]:
from backtest.components.core import ZipKLineIterator1s
from backtest.components import KLineMarket

from datetime import datetime, timezone
import pandas as pd

start_time = datetime(2023, 1, 1, tzinfo=timezone.utc).timestamp() * 1000  # Load one month in advance to do feature engineering
end_time = datetime(2023, 5, 1, tzinfo=timezone.utc).timestamp() * 1000 - 1
data_source = '/home/yite/crypto_data/binance/data'
kline_iterator = ZipKLineIterator1s('BTCUSDT', start_time, end_time, data_source)
df = kline_iterator.to_dataframe()
df = df[["close", "volume", "open_time"]]
df['date'] = pd.to_datetime(df['open_time'], unit='ms')
df.set_index('date', inplace=True)


In [3]:
df

,close,volume,open_time
date,,,
2023-01-01 00:00:00,16542.37,2.03879,1672531200000
2023-01-01 00:00:01,16542.16,2.34046,1672531201000
2023-01-01 00:00:02,16541.62,0.46549,1672531202000
2023-01-01 00:00:03,16541.78,1.55876,1672531203000
2023-01-01 00:00:04,16541.95,2.26832,1672531204000
...,...,...,...
2023-04-30 23:59:55,29223.20,2.99941,1682899195000
2023-04-30 23:59:56,29223.20,0.95576,1682899196000
2023-04-30 23:59:57,29223.20,0.46071,1682899197000


# Feature engineering
The feature engineering is try to derive different aspect,...moving average, volatility, etc
The result should be the state vector as descrbe at the top

In [5]:
import numpy as np

In [6]:
df = df[['close', 'volume']].copy()
df['log_close'] = np.log(df['close'])
df['log_return'] = df['log_close'].diff()
for i in range(0, 19):
    window = 30 * 2 ** i
    df[f'log_return_ma_{window}'] = df['log_return'].rolling(window=window).mean() * 86400 # Daily return of 30 * i  second moving average
    

In [ ]:
import plotly.graph_objects as go

n_col = 10
subset = [f'log_return_ma_{30 * 2 ** i}' for i in range(n_col)]
df_clean = df.dropna(subset=subset)
traces = []
for col in subset:
    trace = go.Scatter(x=df_clean.index, y=df_clean[col], mode='lines', name=col)
    traces.append(trace)
    
# Create the figure object
fig = go.Figure(data=traces)

# Update layout
fig.update_layout(title='Plot of Daily moving average',
                  xaxis_title='timestamp',
                  yaxis_title='return',
                  legend_title='Legend')

# Show plot
fig.show()

In [8]:
n_col = 10
subset = [f'log_return_ma_{30 * 2 ** i}' for i in range(n_col)]
df_clean = df.dropna(subset=subset)

In [11]:
df_clean[subset].to_csv("result.csv")